In [6]:
# %pip install pytesseract pdf2image Pillow
%pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 42.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [52]:
import numpy as np
import cv2
import pytesseract
from pdf2image import convert_from_path
import PIL
from PIL import Image
import os
import re

os.getcwd()

'/Users/danusioguimaraes/Downloads/QUATI'

In [11]:
def preprocess_image(image):
    # Convertendo a imagem para numpy array (necessário para usar OpenCV)
    img_np = np.array(image)

    # Converter a imagem para escala de cinza
    gray = cv2.cvtColor(img_np, cv2.COLOR_BGR2GRAY)
    
    # Aplicar binarização (usando a técnica Otsu)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Converter de volta para imagem PIL
    return Image.fromarray(thresh)

In [23]:
list_prefix = ['ago. 4, Doc 2', 'ago. 4, Doc 3', 'ago. 4, Doc 4', 'ago. 4, Doc 5']
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'
poppler_path = r'/opt/homebrew/bin/'
PIL.Image.MAX_IMAGE_PIXELS = None
custom_config = r'--oem 3 --psm 6'

text_extracted = ""
dict_heads = {}
dict_bodies = {}
for prefix in list_prefix:
    # heads
    try:
        image_path = 'Heads/' + prefix + ' head.png'
        image = Image.open(image_path)

        processed_img = preprocess_image(image)

        text = pytesseract.image_to_string(processed_img, lang='por', config=custom_config)
        
    except:
        text = f"{prefix + ' head.png'} not found."
        print(text)
    
    dict_heads[prefix] = text

    # bodies
    try:
        image_path = 'Bodies/' + prefix + ' body.png'
        image = Image.open(image_path)

        processed_img = preprocess_image(image)

        text = pytesseract.image_to_string(processed_img, lang='por', config=custom_config)

    except:
        msg = f"{prefix + ' body.png'} not found."
        print(msg)
        text = ''
    
    dict_bodies[prefix] = text

    print(f"Text for {prefix} is generated.")
    
for prefix in list_prefix:
    text_extracted += f"\n\n--- Doc: {prefix} ---\n{dict_heads[prefix]} ---\n{dict_bodies[prefix]}"
    print(f"Text for {prefix} added to final file.")

text_extracted += '\nEND'

ago. 4, Doc 2 body.png not found.
Text for ago. 4, Doc 2 is generated.
Text for ago. 4, Doc 3 is generated.
Text for ago. 4, Doc 4 is generated.
Text for ago. 4, Doc 5 is generated.
Text for ago. 4, Doc 2 added to final file.
Text for ago. 4, Doc 3 added to final file.
Text for ago. 4, Doc 4 added to final file.
Text for ago. 4, Doc 5 added to final file.


In [24]:
with open('texto_extraido.txt', 'w') as arquivo_texto:
    arquivo_texto.write(text_extracted)

print("Texto salvo com sucesso.")

Texto salvo com sucesso.


# Output Processing

In [47]:
with open('texto_extraido - v0.txt', 'r') as file:
    text_content = file.read()

print(text_content[100:200])

spostas que você já recebeu
e deve ter preenchido corretamente com todos os dados que lhe são pedido


In [27]:
def extract_substring(text, start_string, end_string):
    start_index = text.find(start_string)
    if start_index == -1:
        return "Start string not found"
    
    start_index += len(start_string)
    
    end_index = text.find(end_string, start_index)
    if end_index == -1:
        return "End string not found"
    
    return text[start_index:end_index]

# Example usage:
text = "This is a sample text with a [start] target substring [end] that we want to extract."
start = "[start]"
end = "[end]"

substring = extract_substring(text, start, end)
print(substring)

 target substring 


In [42]:
starts = ['INSTRUÇÕES', 'A FESTA', 'O TRABALHO', 'A VIAGEM']
ends = ['A FESTA', 'O TRABALHO', 'A VIAGEM', 'END']

print(extract_substring(text_content, starts[1], ends[1]))



Supondo que você está em uma festa. Estão presentes muitas pessoas que você conhece
e outras que você não conhece. Há uma alegre movimentação no salão, com muita comida
e bebida, além de música animada. Então você...
 ---
1. a- aprecia todo este movimento.
b - pensa que talvez fosse melhor ter ficado em casa.
2. a-fica atento para oportunidades de contato que possam To fest
b - presta atenção especial na organização e nos detalhes da festa.
3. a- procura conversar sobre suas idéias e opiniões. . ind
b - prefere conversar sobre o que as pessoas gostam e estão sentindo.
—  b- prefere conversar sobre o que as pessoas gostam eesMo MEO
4. a- presta atenção a tudo e em todos a sua volta.
b - está mais voltado para seu possível incômodo pessoal.
5. a- não fica reparando nos detalhes da festa.
b - atenta especialmente para todos os detalhes e não os esquecerá.
6. a - não hesita em questionar os pontos de vista dos convidados.
b - procura não criticar os outros para não criar um clima desagra

In [108]:
questionario = {}
for i in range(1,len(starts)):
    content = extract_substring(text_content, starts[i], ends[i])
    list_content = content.split(' ---')
    # Regular expression to find the questions (start with a number followed by ".")
    questions_list = re.split(r'(?m)(?=^\d{1,2}\.\s)', list_content[1].strip())

    questions = []
    question_number = 1
    for q in questions_list:  # Assuming 'questions_list' contains the list of questions
        if q.strip():  # Only increment if the string is not empty
            # questions.append(('Q' + str(question_number), [q.strip()]))
            questions.append(('Q' + str(question_number), re.split(r'b\s*-\s*', q.strip().replace('a -', '').replace('a-', ''))))
            question_number += 1

    questionario[starts[i]] = dict(questions)
    # questions = [('Q'+str(j+1), [q.strip()]) for q, j in zip(questions, range(len(questions))) if q]


In [109]:
dict(questions)

{'Q1': ['1.  não pensou muito para escolher o lugar para onde a aonde irá viajar.\n',
  'ponderou uma série de informações antes de decidir-se para onde ir.'],
 'Q2': ['2.  pode alterar o roteiro da viagem no meio do caminho.\n',
  'cumpre o mais possível o roteiro da viagem.'],
 'Q3': ['3.  tem maior interesse em conhecer os fatos do local visitado.\n',
  'tem maior interesse em conhecer o estilo de vida das pessoas que encontrar.'],
 'Q4': ['4.  organiza um roteiro em que passe pela maioria dos pontos turísticos disponíveis.\n',
  'visita somente alguns pontos turísticos mais importantes.'],
 'Q5': ['5. vive mais intensamente a expectativa da viagem.\n',
  'vive mais intensamente o momento da viagem.'],
 'Q6': ['6.  está mais interessado nos conteúdos que irá aprender.\n',
  'está mais interessado nas emoções que irá sentir.'],
 'Q7': ['7.  prefere viajar em um grupo grande de pessoas.\n',
  'prefere viajar em um Pequeno grupo de amigos.'],
 'Q8': ['8.  sente-se mais motivado no plan

In [96]:
print(list_content[0])



Supondo que você vai viajar. É uma viagem de férias e você passará quase um mês
fora de casa. Então você...



In [93]:
print(list_content[1])


1. a- não pensou muito para escolher o lugar para onde a aonde irá viajar.
b - ponderou uma série de informações antes de decidir-se para onde ir.
2. a- pode alterar o roteiro da viagem no meio do caminho.
b - cumpre o mais possível o roteiro da viagem.
3. a- tem maior interesse em conhecer os fatos do local visitado.
b - tem maior interesse em conhecer o estilo de vida das pessoas que encontrar.
4. a - organiza um roteiro em que passe pela maioria dos pontos turísticos disponíveis.
b - visita somente alguns pontos turísticos mais importantes.
5. a-vive mais intensamente a expectativa da viagem.
b - vive mais intensamente o momento da viagem.
6. a - está mais interessado nos conteúdos que irá aprender.
b - está mais interessado nas emoções que irá sentir.
7. a- prefere viajar em um grupo grande de pessoas.
b - prefere viajar em um Pequeno grupo de amigos.
8. a- sente-se mais motivado no planejamento e preparativos para a viagem.
b - sente-se mais motivado quando a viagem se inicia.
9.